In [1]:
import subprocess
from langchain_openai import ChatOpenAI
from langgraph.graph import MessagesState, StateGraph, START
from langgraph.prebuilt import tools_condition, ToolNode
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from IPython.display import Image, display 

def list_usb_devices() -> list:
    """
    Retrieves a list of all USB devices connected to the system using the lsusb command.
    
    Returns:
        list: A list of strings, where each string represents a USB device with its details
              including bus number, device ID, and description.
    """
    devices = []
    result = subprocess.run(['lsusb'], capture_output=True, text=True, check=True)
    for line in result.stdout.strip().split('\n'):
        devices.append(line)
    return devices

# Set up tools
tools = [list_usb_devices]

# Initialize LLM
llm = ChatOpenAI(model="gpt-4")
llm_with_tools = llm.bind_tools(tools)

# System Message
sys_msg = SystemMessage(
    content=(
        "You are a helpful assistant tasked with identifying and managing devices connected to a system. "
        "Users may ask questions about specific devices, such as identifying a particular device, "
        "or general queries about the list of connected devices. Provide precise and detailed responses based on the input list. "
    )
)

# Node
def assistant(state: MessagesState):
    """Process messages in the state and return LLM response."""
    return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}

# Graph setup
builder = StateGraph(MessagesState)
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    tools_condition,
)
builder.add_edge("tools", "assistant")

# Compile graph
agent_graph = builder.compile()

messages = [HumanMessage(content="is there a camera connected?")]
messages = agent_graph.invoke({"messages": messages})


In [5]:
messages = [HumanMessage(content="is there a camera connected?")]
messages = agent_graph.invoke({"messages": messages})

for msg in messages:
    print(msg)

messages


In [3]:
import subprocess
from langchain_openai import ChatOpenAI
from langgraph.graph import MessagesState, StateGraph, START
from langgraph.prebuilt import tools_condition, ToolNode
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

def check_usb_devices(query: str) -> list:
    """
    A function that checks USB devices based on a user query.
    
    Args:
        query (str): The user's question about USB devices
        
    Returns:
        list: List of messages containing the AI's response
    """
    def list_usb_devices() -> list:
        """
        Retrieves a list of all USB devices connected to the system using the lsusb command.
        
        Returns:
            list: A list of strings representing USB devices with their details
        """
        devices = []
        result = subprocess.run(['lsusb'], capture_output=True, text=True, check=True)
        for line in result.stdout.strip().split('\n'):
            devices.append(line)
        return devices

    # Set up tools
    tools = [list_usb_devices]
    
    # Initialize LLM
    llm = ChatOpenAI(model="gpt-4")
    llm_with_tools = llm.bind_tools(tools)
    
    # System Message
    sys_msg = SystemMessage(
        content=(
            "You are a helpful assistant tasked with identifying and managing devices connected to a system. "
            "Users may ask questions about specific devices, such as identifying a particular device, "
            "or general queries about the list of connected devices. Provide precise and detailed responses based on the input list. "
        )
    )
    
    # Node
    def assistant(state: MessagesState):
        """Process messages in the state and return LLM response."""
        return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}
    
    # Graph setup
    builder = StateGraph(MessagesState)
    builder.add_node("assistant", assistant)
    builder.add_node("tools", ToolNode(tools))
    
    # Define edges
    builder.add_edge(START, "assistant")
    builder.add_conditional_edges(
        "assistant",
        tools_condition,
    )
    builder.add_edge("tools", "assistant")
    
    # Compile graph
    agent_graph = builder.compile()
    
    # Create message and get response
    messages = [HumanMessage(content=query)]
    return agent_graph.invoke({"messages": messages})

# Example usage:
result = check_usb_devices("is there a camera connected?")
print(result)

{'messages': [HumanMessage(content='is there a camera connected?', additional_kwargs={}, response_metadata={}, id='5f28224c-ea44-471c-9cec-44c273bfdd93'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Z1TTGaq9jUXcB6YehPXDgNbc', 'function': {'arguments': '{}', 'name': 'list_usb_devices'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 109, 'total_tokens': 118, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-51a5abf1-ae54-47c2-aa29-5118ad14a958-0', tool_calls=[{'name': 'list_usb_devices', 'args': {}, 'id': 'call_Z1TTGaq9jUXcB6YehPXDgNbc', 'type': 'tool_call'}], usage_metadata={'input_tokens': 109, 'output_tokens': 9, 'total_tokens

{'messages': [HumanMessage(content='is there a camera connected?', additional_kwargs={}, response_metadata={}, id='49bb0159-9099-4f78-a373-5bcf0572d95a'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_pOZeTN4d4Cwgus9RMxXrohKm', 'function': {'arguments': '{}', 'name': 'list_usb_devices'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 109, 'total_tokens': 118, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-22baa081-3ba8-47cb-adbf-952f06ea946c-0', tool_calls=[{'name': 'list_usb_devices', 'args': {}, 'id': 'call_pOZeTN4d4Cwgus9RMxXrohKm', 'type': 'tool_call'}], usage_metadata={'input_tokens': 109, 'output_tokens': 9, 'total_tokens': 118, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), ToolMessage(content='["Bus 002 Device 003: ID 8086:0b07 Intel Corp. RealSense D435", "Bus 002 Device 002: ID 0bda:0420 Realtek Semiconductor Corp. 4-Port USB 3.0 Hub", "Bus 002 Device 001: ID 1d6b:0003 Linux Foundation 3.0 root hub", "Bus 001 Device 006: ID 062a:38df MosArt Semiconductor Corp. TIETI UltraSlim KB", "Bus 001 Device 004: ID 1bcf:08a0 Sunplus Innovation Technology Inc. Gaming mouse [Philips SPK9304]", "Bus 001 Device 003: ID 0bda:5420 Realtek Semiconductor Corp. 4-Port USB 2.0 Hub", "Bus 001 Device 002: ID 13d3:3549 IMC Networks Bluetooth Radio", "Bus 001 Device 001: ID 1d6b:0002 Linux Foundation 2.0 root hub"]', name='list_usb_devices', id='0181ca83-9b71-4ccf-8035-93c8d311d9f7', tool_call_id='call_pOZeTN4d4Cwgus9RMxXrohKm'), AIMessage(content='Yes, there is a camera connected to the system. It is an Intel Corp. RealSense D435.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 348, 'total_tokens': 372, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ec02be49-6994-4c9b-8f84-571f8a6897d7-0', usage_metadata={'input_tokens': 348, 'output_tokens': 24, 'total_tokens': 372, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}

In [8]:
import subprocess
from langchain_openai import ChatOpenAI
from langgraph.graph import MessagesState, StateGraph, START
from langgraph.prebuilt import tools_condition, ToolNode
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langgraph.graph.message import AnyMessage, add_messages
from typing import Any, Annotated, TypedDict, Literal

def check_usb_devices(query: str) -> any:
    """
    A function that checks USB devices based on a user query.
    
    Args:
        query (str): The user's question about USB devices
        
    Returns:
        str: The final AI response message content
    """

    # This is the default state same as "MessagesState" TypedDict but allows us accessibility to custom keys
    class GraphsState(TypedDict):
        messages: Annotated[list[AnyMessage], add_messages] 
        # Custom keys for additional data can be added here such as - conversation_id: str

    def list_usb_devices() -> list:
        """
        Retrieves a list of all USB devices connected to the system using the lsusb command.
        
        Returns:
            list: A list of strings representing USB devices with their details
        """
        devices = []
        result = subprocess.run(['lsusb'], capture_output=True, text=True, check=True)
        for line in result.stdout.strip().split('\n'):
            devices.append(line)
        return devices

    # Set up tools
    tools = [list_usb_devices]
    
    # Initialize LLM
    llm = ChatOpenAI(model="gpt-4o")
    llm_with_tools = llm.bind_tools(tools)
    
    # System Message
    sys_msg = SystemMessage(
        content=(
            "You are a helpful assistant tasked with identifying and managing devices connected to a system. "
            "Users may ask questions about specific devices, such as identifying a particular device, "
            "or general queries about the list of connected devices. Provide precise and detailed responses based on the input list. "
        )
    )
    
    # Node
    def assistant(state):
        """Process messages in the state and return LLM response."""
        return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}
    
    # Graph setup
    builder = StateGraph(MessagesState)
    builder.add_node("assistant", assistant)
    builder.add_node("tools", ToolNode(tools))
    
    # Define edges
    builder.add_edge(START, "assistant")
    builder.add_conditional_edges(
        "assistant",
        tools_condition,
    )
    builder.add_edge("tools", "assistant")
    
    # Compile graph
    agent_graph = builder.compile()
    
    # Create message and get response
    messages = [HumanMessage(content=query)]
    result = agent_graph.invoke({"messages": messages})
    
    # Extract the final AI message content
    final_message = [msg for msg in result["messages"] if isinstance(msg, AIMessage)][-1]
    return final_message.content

# Example usage:
response = check_usb_devices("is there a camera connected?")
print(response)  # Will print only the final AI response


Yes, there is a camera connected to the system. It is the "Intel Corp. RealSense D435" camera.
